##**Install Langchain and Google Gemini related libraries**

In [ ]:
!pip install -q -U langchain
!pip install -q -U google-generativeai langchain-google-genai

##**Get Gemini Key from Secrets **
Set GEMINI_KEY secret key at Google Colab and get that here to runn Google Gemini LLM. You can get Google Gemini Key from following link https://makersuite.google.com/app/apikey

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_KEY')

##**Run Simple LLM Prompt**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=True)

chain = prompt | llm

In [ ]:
resp = chain.invoke({"input": "how can langsmith help with testing?"})
print(resp)

##**Run LLM with OutputParser**

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
resp = chain.invoke({"input": "how can langsmith help with testing?"})
print(resp)

##**Run Retrieval Augumented Generation with LangChain and Gemini**
We are using beautifulsoup4 to scrap web data and use that as external source to augment to LLM

In [ ]:
!pip install beautifulsoup4

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [ ]:
!pip install -q -U sentence-transformers
from langchain.embeddings import HuggingFaceEmbeddings
# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
!pip install faiss-gpu

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

##**Run Converstation Retrieval chain with history**
The provided way in LangChain quickstart guide throws error when we use Google Gemini with LangChain. There might be support issue of Langchain with Google Gemini. This is alternative method.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2'))

In [ ]:
from langchain.chains import ConversationalRetrievalChain

# Connect query to FAISS index using a retriever
retriever = vector.as_retriever()

qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever,return_source_documents=True)
print(retriever)

In [ ]:
import sys
chat_history = []
while True:
    query = input('Prompt: ')
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))